# Generate Synthetic Data with Conversation Starters

Contoso Zoo is developing an AI-powered smart guide app to enhance visitors' experiences. The app aims to answer questions about wildlife. To evaluate the responses from the app, we need to create a comprehensive synthetic question-answer dataset that covers various aspects of wildlife, including animal behavior, habitat preferences, and dietary needs. <br>

In this exercise, you will generate a synthetic question-answer dataset for the Contoso Zoo app. This dataset will not only include standard question-answer pairs but also incorporate conversation starters to simulate contextually relevant interactions. These conversation starters will allow for pre-specified, repeatable user turns in a conversation, which is crucial for evaluating the AI's performance across different scenarios.


## Install packages

To begin, we'll install the packages that are necessary for completing the simulation. The `Simulator` class is within the `simulator` package of the Azure AI Evaluation SDK. With security in mind, we'll authenticate with Azure services using `azure-identity`. And finally, since we'll be interacting with OpenAI's services via Azure, the `openai` package will come in handy!

In [ ]:
%pip install azure-ai-evaluation azure-identity openai

## Import packages

We'll now import the `Simulator` class alongside several additional packages and modules that'll be necessary for running the simulation.

In [ ]:
import os
import json
from azure.ai.evaluation.simulator import Simulator
from azure.identity import DefaultAzureCredential
from typing import List, Dict, Any, Optional
from openai import AzureOpenAI
from pathlib import Path

## Set environment variables to create an instance of the simulator

We'll now set the environment variables that'll be required to create an instance of the `Simulator` class. You'll need the following:

- Azure OpenAI endpoint
- Azure OpenAI API Key
- Azure deployment
- Azure OpenAI API version

You can locate your **Azure OpenAI endpoint** and **Azure OpenAI API Key** by navigating to **Models + endpoints**, selecting the model, and copying the respective credentials for your model deployment.

The **OpenAI API version** is available in the [API version lifecycle](https://learn.microsoft.com/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release) document.

In [ ]:
os.environ['AZURE_OPENAI_ENDPOINT'] = 'Your OpenAI endpoint'
os.environ['AZURE_OPENAI_API_KEY'] = 'Your OpenAI API key'
os.environ['AZURE_OPENAI_DEPLOYMENT'] = 'Your deployment'
os.environ['OPENAI_API_VERSION'] = 'The OpenAI API version'

## Configure the model_config

The `model_config` is necessary as it's a required parameter when creating an instance of the `Simulator` class. Let's configure the `model_config` with the following:

- Azure OpenAI endpoint
- Azure OpenAI API key
- Azure deployment

In [ ]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

should_cleanup: bool = False

## Create an instance of the Simulator class

We now need to create an instance of the `Simulator` class which will later be used to run the simulator. We'll need to initialize the `simulator` object with the `model_config`.

In [ ]:
simulator = Simulator(model_config)

## Create a call to your application

We'll now create a function that'll create a call to your application which should take a user query and return a response. 

The call to the AI model includes parameters which specifies the model, messages, and additional settings which impact the model's response. 

We'll later send a query to the AI model and retrieve the response. Once the response is retrieved, we'll convert the completion object to a dictionary and extract the message before returning the message content.

In [ ]:
def call_to_your_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors
    deployment = os.environ.get("AZURE_DEPLOYMENT")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        api_key=os.environ.get("AZURE_API_KEY"),
    )
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    message = completion.to_dict()["choices"][0]["message"]
    # change this to return the response from your application
    return message["content"]

## Create a callback

We'll now create a `callback` which will handle incoming messages, process them using the app, and format the response to follow the OpenAI chat protocol format. 

The `callback` function will extract the latest message from the conversation history, reset the context, and then call the app to get a response based on the latest message. 

Afterwards, the response will be formatted to follow the OpenAI chat protocol format, and then appended to the conversation history. We'll retun the updated conversation history.

In [ ]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_your_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

## Create conversation starters

When defining the conversation starters, we want to provide examples that are relevant to a real-world use-case. Here we provide 2 conversation starters for two separate conversations.

The first conversation simulates a person discussing lions with the app. The second conversation simulates a parent inquiring about birds with the app. Both conversations are stored within a `conversations_turns` list.

**Note**: Later when you run the simulator, if you consistently exceed quota, remove one of the simulations from the `conversation_turns` list and run the subsequent cells once more.

In [ ]:
conversation_turns = [
    # simulation 1
    [
        "I'm standing in front of the lion enclosure, and I've never seen a real lion before!",  # simulator conversation starter (turn 1)
        "Wow, the lion is much bigger than I expected! How much do they typically weigh?",  # conversation turn 2,
        "I notice the lion is just lying there. Do they sleep a lot? What do they do most of the day?",  # conversation turn 3
    ],
    # simulation 2
    [
        "My child is fascinated by the colorful birds in the aviary. She keeps asking about their feathers.",
        "My daughter wants to know why some birds have such bright colors while others are more dull. Can you explain?",
        "She's now wondering if birds can change the color of their feathers like chameleons change their skin color.",
    ],
]

## Create a call to the simulator

Let's now create a call to run the simulation and process the conversation turns.

In [ ]:

outputs = await simulator(
    target=callback,
    conversation_turns=conversation_turns,
    max_conversation_turns=3,
)

## Save the generated data

Finally, save the generated data to a `.json` file.

In [ ]:
output_file = Path("conversation_starter_output.json")
with output_file.open("a") as f:
    json.dump(outputs, f)

## Delete resources

If you've finished exploring Azure AI Services, delete the Azure resource that you created during the workshop.

**Note**: You may be prompted to delete your deployed model(s) before deleting the resource group.